In [2]:
#https://realpython.com/python-send-email/#sending-fancy-emails

import smtplib, email

from email import encoders
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
from datetime import date, timedelta

In [3]:
# Je crée un échantillon test, mettre True si mailing_week_0 n'existe pas
test_ = False
if test_:
    df_test = pd.DataFrame({'entreprise' : ['DylanCorp.', 'ZakaryCorp.'],
                        'presentation' : ['Monsieur Anton,', 'Monsieur Yousfi,'],
                        'mail' : ['dylan_a94@hotmail.fr', 'zakary.zy@gmail.com'],
                        'date_send' : ['week_0', 'week_0'],
                        'civilite' : ['Monsieur', 'Monsieur'],
                        'Nom' : ['Anton', 'Yousfi'],
                        'Prenom' : ['Dylan', 'Zakary'],
                        'telephone' : ['0678797475', '0612345678']})
    df_test.to_pickle('mailing_week_0.pkl')
    
    
zak_test = True #Zak Test permet de tester week_0 et de voir si ton script marche bien avec le pickle contenant nos mails

if zak_test:
    start, end = date(2020, 10, 10), date(2020, 12, 5)
    days = (start + timedelta(days=i) for i in range((end - start).days + 1))
    dates = [d for d in days if d.weekday() in [5]]
else: #Vrai truc
    start, end = date(2020, 10, 10), date(2020, 11, 30)
    days = (start + timedelta(days=i) for i in range((end - start).days + 1))
    dates = [d for d in days if d.weekday() in [0]]

In [4]:
#Nombre de semaines
periode = 8

pickles_existed = True

if pickles_existed == False:
    #Lecture du fichier xlsx
    df_final = pd.read_excel('df_final_correction.xlsx', index_col=0)

    #drop les lignes quand on a un doublon au niveau du mail
    df_final = df_final.drop_duplicates('mail', keep='first')

    # shuffle le dataframe
    df_final_shuffle = df_final.sample(frac=1, random_state=42)

    #récupère le nombre de lignes que doit contenir chaque df serialisé
    nb_contact_per_pickle = round(df_final_shuffle.shape[0]/periode)

    #Création des noms "week_" allant de 1 jusqu'à la variable periode (ici 8)
    weeks = ['week_' + str(i) for i in range(1,periode + 1)]

    #Création de mes df split en 8, puis pickle
    for week, i in zip(weeks, range(0,df_final_shuffle.shape[0], nb_contact_per_pickle)):
        if week != weeks[-1]:
            df_split = df_final_shuffle.iloc[i:i+77]
        else:
            df_split = df_final_shuffle.iloc[i::]

        df_split.drop('is_send', 1, inplace = True)
        df_split.date_send.fillna(week, inplace = True)
        df_split.to_pickle('mailing_' + week + '.pkl')

try:
    date_today = date.today()

    for num_week, date_of_week in enumerate(dates, start = 0):
        if date_of_week == date_today:
            df_to_send = pd.read_pickle('mailing_week_' + str(num_week) + '.pkl')
            break
        else:
            continue

    print("Le fichier qui va être envoyé concerne", df_to_send.date_send.iloc[0])
    print("Si c'est la bonne semaine alors on peut continuer, sinon il faut s'arreter !!")
except:
    print("Les pickles n'existent pas, il faut mettre l'argument pickles_existed à True")

Le fichier qui va être envoyé concerne week_0
Si c'est la bonne semaine alors on peut continuer, sinon il faut s'arreter !!


## Connexion au compte Gmail

In [2]:
# Mail and password to connect the SMTP server
sender_email = "zakary.yousfi@gmail.com"
password = ""

# Set up the SMTP server
s = smtplib.SMTP(host='smtp.gmail.com', port=587)
s.starttls()
s.login(sender_email, password)

(235, b'2.7.0 Accepted')

## Traitement de la BDD

In [3]:
#import des contacts de la semaine

d = {'entreprise': ["DylanCorp.","ZakCorp."], 'contact': ["Dylan Anton","Zakary Yousfi"], 'mail': ["dylan_a94@hotmail.fr","zakary.zy@gmail.com"], 'presentation': ["Monsieur Anton,","Monsieur Yousfi,"], 'entreprise contactee': [0,0], "date d'envoi" : ['','']}
df = pd.DataFrame(data=d)

In [18]:
#import du pickle de la semaine
#df1 = pd.read_excel('df_final_correction.xlsx', index_col= 0)

## Creation du Mail

In [4]:
for i in range(0,2):
    if df['entreprise contactee'][i] == 0:
        
        # Create mail
        message = MIMEMultipart("alternative")

        # Import attachment in same directory as script

        CV = "CV YOUSFI ZAKARY DEV UNITY.pdf" 
        LM = "LM YOUSFI ZAKARY DEV UNITY.pdf"

        #Open the CV an LM

        with open(CV, "rb") as attachment1:
            # Add file as application/octet-stream
            # Email client can usually download this automatically as attachment
            part1 = MIMEBase("application", "octet-stream")
            part1.set_payload(attachment1.read())

        with open(LM, "rb") as attachment2:
            # Add file as application/octet-stream
            # Email client can usually download this automatically as attachment
            part2 = MIMEBase("application", "octet-stream")
            part2.set_payload(attachment2.read())
    
        # Encode file in ASCII characters to send by email    
        encoders.encode_base64(part1)
        encoders.encode_base64(part2)

        # Add header as key/value pair to attachment part
        part1.add_header(
            "Content-Disposition",
            f"attachment; filename= {CV}",
        )
        part2.add_header(
            "Content-Disposition",
            f"attachment; filename= {LM}",
        )

        nom_presentation = str(df['presentation'][i])
        nom_entreprise = str(df['entreprise'][i])
        receiver_email = str(df['mail'][i])

        #subject of the mail
        subject = "Zakary - alternant développeur Unity"

        # Create the plain-text and HTML version of your message
        text = """\
        {}


        Je me présente Zakary YOUSFI et je vous propose ma candidature pour une alternance en tant que développeur Unity.

        J'ai de l'expérience dans le domaine de la programmation, acquise en autodidacte et au cours de mon Master Data Analyst. Si un profil créatif et dynamique vous intéresse, vous pouvez visiter mon site internet https://zakary.co.

        Vous trouverez en pièce jointe mon CV et ma lettre de motivation. Je serais ravi d'échanger avec vous afin de détailler mon parcours et d’évoquer ma candidature chez {}.


        Bien cordialement,


        Zakary Yousfi

        +33 6 67 99 12 43



        Aide exceptionnelle aux employeurs qui recrutent en apprentissage: https://travail-emploi.gouv.fr/formation-professionnelle/entreprise-et-alternance/aide-exceptionnelle-apprentissage
        """.format(nom_presentation,nom_entreprise)

        html = """\
        <html>
          <body>
            <p>{}</p>
            <p>Je me pr&eacute;sente Zakary YOUSFI et je vous propose ma candidature pour une alternance en tant que d&eacute;veloppeur Unity.</p>
            <p>J&apos;ai de l&apos;exp&eacute;rience dans le domaine de la programmation, acquise en autodidacte et au cours de mon Master Data Analyst. Si un profil cr&eacute;atif et dynamique vous int&eacute;resse, vous pouvez visiter mon site internet <a href="https://zakary.co" rel="noopener noreferrer" target="_blank">https://zakary.co</a>.</p>
            <p>Vous trouverez en pi&egrave;ce jointe mon CV et ma lettre de motivation. Je serais ravi d&apos;&eacute;changer avec vous afin de d&eacute;tailler mon parcours et d&rsquo;&eacute;voquer ma candidature chez {}.</p>
            <p>Bien cordialement,</p>
            <p><br></p>
            <p>Zakary Yousfi</p>
            <p>+33 6 67 99 12 43</p>
            <p><br></p>
            <p>Aide exceptionnelle aux employeurs qui recrutent en apprentissage: <a href="https://travail-emploi.gouv.fr/formation-professionnelle/entreprise-et-alternance/aide-exceptionnelle-apprentissage" rel="noopener noreferrer" target="_blank">https://travail-emploi.gouv.fr/formation-professionnelle/entreprise-et-alternance/aide-exceptionnelle-apprentissage</a></p>
            </body>
        </html>
        """.format(nom_presentation,nom_entreprise)

        #add attachment
        message.attach(part1)
        message.attach(part2)

        #Objet
        message["Subject"] = subject

        #De
        message["From"] = sender_email

        #A
        message["To"] = receiver_email

        #Add plain text
        message.attach(MIMEText(text, "plain"))
        # Add HTML
        message.attach(MIMEText(html, "html"))

        #send mail
        s.sendmail(sender_email, receiver_email, message.as_string())

        #save data
        #date d'envoi
        df['entreprise contactee'][i] = str(date.today())
        #variable entreprise contactee = 1
        df['date d\'envoi'][i] = 1

        #print message 
#        print("Mail sent to " + receiver_email)

C:\Users\zakar\anaconda3\lib\site-packages\ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\zakar\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Mail sent to dylan_a94@hotmail.fr
Mail sent to zakary.zy@gmail.com


In [15]:
#mise a jour du fichier excel (changer le nom du fichier pour qu'il soit le meme que le fichier importer)
#df.to_excel('postulobot - {}.xlsx'.format(str(date.today())))